#Ejemplo CPU
En este documento se evidencia la forma en la que se replica el ejemplo presentado por Sotaquirá en su video introductorio a Google Colab. Este cuaderno corresponde al caso en el que se usa la CPU para programar el modelo.

In [ ]:
import numpy as np
import os
import gzip
def load_mnist(ruta, tipo='train'):

    ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo) 
    ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)

    with gzip.open(ruta_categorias, 'rb') as rut_cat:
     etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)

    with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
     imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas),784)

    return imagenes, etiquetas



#Accediendo a Google Drive

In [17]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

ruta='gdrive/My Drive/Colab Notebooks/Tarea 1/'

X_train, Y_train = load_mnist(ruta, tipo='train')
X_test, Y_test = load_mnist(ruta, tipo='test')

Mounted at /content/gdrive


#Ajustando los datos

In [ ]:
X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

#Implementando el modelo

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print('Versión de TensorFlow: ' +tf.__version__)

tf.random.set_seed(200)

Versión de TensorFlow: 2.8.0


# Creación del modelo

In [14]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_3 (Batc  (None, 28, 28, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_4 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                      

#Entrenamiento con CPU

In [18]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


#Generando el Checkpoint

In [19]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='gdrive/My Drive/Colab Notebooks/Tarea 1/Codigos y checkpoints/Checkpoint CPU')

In [13]:
import timeit

def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1,callbacks=[model_checkpoint_callback])
  
  return None

cpu_time = timeit.timeit('entrenamiento_cpu()', number=1, setup='from __main__ import entrenamiento_cpu')

Epoch 1/2
468/468 [==============================] - 803s 2s/step - loss: 0.3191 - accuracy: 0.8843 - val_loss: 0.2834 - val_accuracy: 0.9003
Epoch 2/2
468/468 [==============================] - 792s 2s/step - loss: 0.2804 - accuracy: 0.8989 - val_loss: 0.2835 - val_accuracy: 0.9023


In [15]:
print('Tiempo de entrenamiento ' + str(cpu_time) + ' segundos')

Tiempo de entrenamiento 1642.0012123949996 segundos
